In [1]:
import pandas as pd
import pickle
import numpy as np
from brainflow.board_shim import BoardShim, BrainFlowInputParams, LogLevels, BoardIds
from brainflow.data_filter import DataFilter, FilterTypes
import matplotlib
import os
import torch
import pytorch_lightning as pl
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from pytorch_lightning.loggers import WandbLogger

import cv2
from torch.utils.data import DataLoader
import wandb

matplotlib.use('Agg')
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
%matplotlib inline
scaler = StandardScaler()

In [2]:
wandb.login()
api_key = 'a45abb01f9556b57620ce77c8984452bee7a8772'
board_id = 38
sf = 256

wandb: Currently logged in as: nickhartono. Use `wandb login --relogin` to force relogin


In [3]:
class LitAutoEncoder6(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.encoder = torch.nn.Sequential(
            torch.nn.Conv2d(2, 8, 3, stride=2, padding=1),
            torch.nn.BatchNorm2d(8),
            torch.nn.ReLU(True),
            torch.nn.Conv2d(8, 16, 3, stride=2, padding=1),
            torch.nn.BatchNorm2d(16),
            torch.nn.ReLU(True),
            torch.nn.Conv2d(16, 32, 3, stride=2, padding=0),
            torch.nn.ReLU(True),
            torch.nn.Conv2d(32, 64, 3, stride=1, padding=0),
            torch.nn.ReLU(True),
            torch.nn.BatchNorm2d(64),
            torch.nn.Conv2d(64, 64, 3, stride=2, padding=1),
            torch.nn.ReLU(True),
        )
        self.decoder = torch.nn.Sequential(
            torch.nn.ConvTranspose2d(64, 64, 3, stride=2,
            padding=1, output_padding=1),
            torch.nn.BatchNorm2d(64),
            torch.nn.ReLU(True),
            torch.nn.ConvTranspose2d(64, 32, 3, stride=1,
            padding=0, output_padding=0),
            torch.nn.ReLU(True),
            torch.nn.ConvTranspose2d(32, 16, 3, stride=2,
            padding=0, output_padding=0),
            torch.nn.ReLU(True),
            torch.nn.BatchNorm2d(16),
            torch.nn.ConvTranspose2d(16, 8, 3, stride=2,
            padding=1, output_padding=1),
            torch.nn.BatchNorm2d(8),
            torch.nn.ReLU(True),
            torch.nn.ConvTranspose2d(8, 2, 3, stride=2,
            padding=1, output_padding=1)
        )

    def forward(self, x):
      embedding = self.encoder(x)
      return embedding

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # it is independent of forward
        x = batch
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = torch.nn.functional.mse_loss(x_hat, x)
        # Logging to TensorBoard (if installed) by default
        self.log("train_loss", loss, logger=True, on_epoch=True)
        return loss

    def validation_step(self, val_batch, batch_idx):
        x = val_batch
        z = self.encoder(x)
        x_hat = self.decoder(z)
        val_loss = torch.nn.functional.mse_loss(x_hat, x)
        self.log("val_loss", val_loss, logger=True, on_epoch=True)

    def test_step(self, batch, batch_idx):
        # this is the test loop
        x = batch
        z = self.encoder(x)
        x_hat = self.decoder(z)
        test_loss = torch.nn.functional.mse_loss(x_hat, x)
        self.log("test_loss", test_loss, logger=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [4]:
autoencoder = LitAutoEncoder6().load_from_checkpoint('D:\\Nicko\\TUGAS_AKHIR\\Classifier\\model_latent_32640_2channel.ckpt')
autoencoder.eval()

LitAutoEncoder6(
  (encoder): Sequential(
    (0): Conv2d(2, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2))
    (7): ReLU(inplace=True)
    (8): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (9): ReLU(inplace=True)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (12): ReLU(inplace=True)
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runn

In [5]:
pkl_dir = 'D:\\Nicko\\TUGAS_AKHIR\\Dataset_new\\Dataset_TA_pkl\\'

In [6]:
img_dir = 'D:\\Nicko\\TUGAS_AKHIR\\Dataset_new\\Dataset_TA_img\\'
list_file_dir = 'D:\\Nicko\\TUGAS_AKHIR\\Dataset_new\\Dataset_TA\\CLASSIFIER_NEW_DIR.csv'
destination_dir = 'D:\\Nicko\\TUGAS_AKHIR\\Dataset_new\\Dataset_TA_pkl\\LATENT_32640_2CHANNEL\\'
eeg_names = ['Fp1', 'Fp2']
file_lists = pd.read_csv(list_file_dir, header=None)

In [7]:
path_list = []
label_list = []

In [8]:
ctr = 1
for i in range(len(file_lists)):
  control = img_dir + file_lists.iloc[i, 0][:-4].replace('/','\\') + '\\'
  relevant = img_dir + file_lists.iloc[i, 1][:-4].replace('/','\\') + '\\'
  label = file_lists.iloc[i, 2]

  # read image
  combined = []
  control_img = []
  relevant_img = []

  for j in eeg_names:
    temp_control = cv2.imread(control + j + '.png', cv2.IMREAD_GRAYSCALE)
    temp_relevant = cv2.imread(relevant + j + '.png', cv2.IMREAD_GRAYSCALE)
    control_img.append(temp_control)
    relevant_img.append(temp_relevant)
  control_img = torch.tensor(np.array(control_img)).type(torch.FloatTensor)
  relevant_img = torch.tensor(np.array(relevant_img)).type(torch.FloatTensor)
  with torch.no_grad():
      encoded_control = autoencoder(control_img[None, :])
      encoded_relevant = autoencoder(relevant_img[None, :])
  combined.append(encoded_control.detach().numpy())
  combined.append(encoded_relevant.detach().numpy())
  combined = np.array(combined)

  # save to pickle
  user_id = control[56:58]
  folder = destination_dir + 'USER' + user_id
  if not os.path.exists(folder):
    os.mkdir(folder)
    ctr = 1

  filename = folder + '\\' + 'PAIR_' + str(ctr).zfill(2) + '.pickle'
  print(filename)

  savefile = open(filename, 'ab')
  pickle.dump(combined, savefile)
  savefile.close()

  ctr += 1
  path_list.append(filename)
  label_list.append(label)

D:\Nicko\TUGAS_AKHIR\Dataset_new\Dataset_TA_pkl\LATENT_32640_2CHANNEL\USER01\PAIR_01.pickle
D:\Nicko\TUGAS_AKHIR\Dataset_new\Dataset_TA_pkl\LATENT_32640_2CHANNEL\USER01\PAIR_02.pickle
D:\Nicko\TUGAS_AKHIR\Dataset_new\Dataset_TA_pkl\LATENT_32640_2CHANNEL\USER01\PAIR_03.pickle
D:\Nicko\TUGAS_AKHIR\Dataset_new\Dataset_TA_pkl\LATENT_32640_2CHANNEL\USER01\PAIR_04.pickle
D:\Nicko\TUGAS_AKHIR\Dataset_new\Dataset_TA_pkl\LATENT_32640_2CHANNEL\USER01\PAIR_05.pickle
D:\Nicko\TUGAS_AKHIR\Dataset_new\Dataset_TA_pkl\LATENT_32640_2CHANNEL\USER01\PAIR_06.pickle
D:\Nicko\TUGAS_AKHIR\Dataset_new\Dataset_TA_pkl\LATENT_32640_2CHANNEL\USER01\PAIR_07.pickle
D:\Nicko\TUGAS_AKHIR\Dataset_new\Dataset_TA_pkl\LATENT_32640_2CHANNEL\USER02\PAIR_01.pickle
D:\Nicko\TUGAS_AKHIR\Dataset_new\Dataset_TA_pkl\LATENT_32640_2CHANNEL\USER02\PAIR_02.pickle
D:\Nicko\TUGAS_AKHIR\Dataset_new\Dataset_TA_pkl\LATENT_32640_2CHANNEL\USER02\PAIR_03.pickle
D:\Nicko\TUGAS_AKHIR\Dataset_new\Dataset_TA_pkl\LATENT_32640_2CHANNEL\USER02\PAI

In [9]:
new_list = []
for i in path_list:
    new_list.append(i.replace('/','\\'))

In [10]:
len(new_list)

216

In [11]:
len(path_list)

216

In [12]:
len(label_list)

216

In [13]:
df = {}
df['path'] = new_list
df['label'] = label_list
df = pd.DataFrame(df)

In [14]:
df.to_csv(destination_dir + 'path_and_label.csv')